# Optimizing Renewable Energy Installations Geographically
_How can renewable energy installations be geographically optimized to maximize output and minimize cost?_

Before diving deep into our research questionw we want to narrow down the number of countries taking part in it so as to have better data cleaning, analysis, mapping, and precise, clear insights: we value clarity, focus, and reasoning.
We will select 5 countries based on the following three metrics:
1. Renewable Resource Potential: the core geographic variable — solar irradiance, wind speeds and land use patterns are all spatially distributed.
2. Cost-Related Indicators: even if a place has great resources, costs can vary with location due to proximity to roads and grid, terrain difficulty, labor/infrastructure availability: we want locations where energy is not only abundant but also cheap to install and deliver.
3. Energy Demand & Access: wile not directly geographic, access to electricity and consumption patterns are spatially mapped, often down to region or village level. Optimizing means placing energy where it’s not only cheap and abundant, but most needed.
These form the geographic optimization triangle: "where it’s best to produce + easiest to build + most needed."

This can help us trasformate our research question in a case study: _"We selected Country A, B, and C as case studies to demonstrate how geographic optimization can vary based on resource potential, grid infrastructure, and demand"_.



### World - Public Investments

Data was downloaded from https://energydata.info/dataset/world-public-investments-2019-million-usd-by-country-area-technology-and-year-2020-2000
Dataset columns: ['Country', 'Year', 'Bioenergy', 'Fossil fuels', 'Geothermal energy', 'Marine energy', 'Multiple renewables*', 'Nuclear', 'Other non-renewable energy', 'Other renewable energy', 'Renewable hydropower', 'Solar energy', 'Wind energy']

For our reseach we want to filter out all the countries for which we don't have data in all our datasets of interest and use only the following data: ['Country', 'Year', 'Fossil fuels', 'Multiple renewables*', 'Solar energy', 'Wind energy']

In [3]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd

df = pd.read_excel(r"C:\Users\lucre\Downloads\World_Public_Investments.xlsx")

df.columns = ["Country", "Technology", "Year", "Investment"]

df["Country"] = df["Country"].fillna(method="ffill")
df["Technology"] = df["Technology"].fillna(method="ffill")

df["Investment"] = df["Investment"].replace("..", pd.NA)

df["Investment"] = df["Investment"].astype(str).str.replace(",", ".")
df["Investment"] = pd.to_numeric(df["Investment"], errors="coerce")

df = df.dropna(subset=["Year", "Investment"])

C:\Users\lucre\AppData\Local\Temp\ipykernel_17288\3289059684.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Country"] = df["Country"].fillna(method="ffill")
C:\Users\lucre\AppData\Local\Temp\ipykernel_17288\3289059684.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Technology"] = df["Technology"].fillna(method="ffill")


In [9]:
# Pivot so each energy type is a column
df_pivot = df.pivot_table(
    index=["Country", "Year"],
    columns="Technology",
    values="Investment",
    aggfunc="sum"  # in case there are duplicates
).reset_index()

df_pivot.columns.name = None

In [19]:
print(df_pivot.head(10))

       Country    Year  Bioenergy  Fossil fuels  Geothermal energy  \
0  Afghanistan  2000.0        NaN           NaN                NaN   
1  Afghanistan  2001.0        NaN           NaN                NaN   
2  Afghanistan  2002.0        NaN         11.72                NaN   
3  Afghanistan  2003.0        NaN          0.10                NaN   
4  Afghanistan  2004.0        NaN         60.09                NaN   
5  Afghanistan  2005.0        NaN         72.56                NaN   
6  Afghanistan  2006.0        NaN           NaN                NaN   
7  Afghanistan  2007.0        NaN           NaN                NaN   
8  Afghanistan  2008.0        NaN           NaN                NaN   
9  Afghanistan  2009.0        NaN           NaN                NaN   

   Marine energy  Multiple renewables*  Nuclear  Other non-renewable energy  \
0            NaN                   NaN      NaN                         NaN   
1            NaN                   NaN      NaN                        

In [13]:
print(df_pivot.columns.tolist())

['Country', 'Year', 'Bioenergy', 'Fossil fuels', 'Geothermal energy', 'Marine energy', 'Multiple renewables*', 'Nuclear', 'Other non-renewable energy', 'Other renewable energy', 'Renewable hydropower', 'Solar energy', 'Wind energy']


In [18]:
selected_columns = [
    'Country', 'Year', 'Fossil fuels', 'Multiple renewables*', 'Solar energy', 'Wind energy'
]

filtered_investments = df_pivot[selected_columns]
filtered_investments.head(5)

,Country,Year,Fossil fuels,Multiple renewables*,Solar energy,Wind energy
0,Afghanistan,2000.0,NaN,NaN,NaN,NaN
1,Afghanistan,2001.0,NaN,NaN,0.1,NaN
2,Afghanistan,2002.0,11.72,NaN,NaN,NaN
3,Afghanistan,2003.0,0.10,25.68,NaN,NaN
4,Afghanistan,2004.0,60.09,NaN,NaN,NaN


## World - Regulatory Indicators for Sustainable Energy
The RISE dataset, downloadable at https://energydata.info/dataset/world-regulatory-indicators-sustainable-energy-2016, covers 111 countries across the developed and developing worlds, which together represent over 90% of global population, GDP and energy consumption. With 28 indicators, 85 sub-indicators and 158 data points per country, RISE helps policy makers to understand how they are doing, compare across countries, learn from peer groups, and identify priority actions for the future.

For each country analysed and for each indicator there is a score that determines how well that country is doing in that specific field (eg. energy efficiency). We can use the scores for our evaluation by incorporating them into our analysis on the countries photovoltaic and wind energy potential.

In [6]:
RISE_df = pd.read_excel(r"C:\Users\lucre\Downloads\Datasets and maps\RISE_dataset.xlsx", header = 1)
RISE_df.columns = RISE_df.columns.str.strip()

In [7]:
RISE_subset = RISE_df[[
    'Country',
    'Income group',
    'Energy Access score',
    'Renewable Energy score',
    'Energy Efficiency score',
    'Overall score',
    'Indicator 1:  Existence and monitoring of officially approved electrification plan',
    'Indicator 3: Framework for grid electrification',
    'Indicator 6:  Consumer affordability of electricity',
    'Indicator 1: National energy efficiency planning',
    'Indicator 8: Minimum energy efficiency performance standards',
    'Indicator 1: Legal framework for renewable energy',
    'Indicator 2: Planning for renewable energy expansion',
    'Indicator 3: Incentives and regulatory support for renewable energy',
    'Indicator 5: Network connection and pricing'
]]

RISE_subset.rename(columns={
    'Energy Access score': 'Access Score',
    'Renewable Energy score': 'Renewables Score',
    'Energy Efficiency score': 'Efficiency Score',
    'Overall score': 'Overall Score',
    'Indicator 1:  Existence and monitoring of officially approved electrification plan': 'Electrification Plan Monitoring',
    'Indicator 3: Framework for grid electrification': 'Grid Electrification Framework',
    'Indicator 6:  Consumer affordability of electricity': 'Electricity Affordability',
    'Indicator 1: National energy efficiency planning': 'Efficiency Planning',
    'Indicator 8: Minimum energy efficiency performance standards': 'Efficiency Standards',
    'Indicator 1: Legal framework for renewable energy': 'Renewable Energy Legal Framework',
    'Indicator 2: Planning for renewable energy expansion': 'Renewables Expansion Planning',
    'Indicator 3: Incentives and regulatory support for renewable energy': 'Renewables Incentives',
    'Indicator 5: Network connection and pricing': 'Network Connection & Pricing'
}, inplace=True)

print(RISE_subset.head())

       Country         Income group  Access Score  Renewables Score  \
0  Afghanistan           Low income     23.992691         26.934385   
1      Algeria  Upper middle income    100.000000         51.101032   
2       Angola  Upper middle income     48.298507         17.361071   
3    Argentina          High income    100.000000         52.781026   
4      Armenia  Lower middle income    100.000000         63.071833   

   Efficiency Score  Overall Score  Electrification Plan Monitoring  \
0         17.898479      22.941852                                0   
1         55.987654      69.029562                              100   
2         17.749669      27.803083                               80   
3         43.946208      65.575745                              100   
4         41.914683      68.328839                              100   

   Grid Electrification Framework  Electricity Affordability  \
0                       33.333333                       50.0   
1                 

C:\Users\lucre\AppData\Local\Temp\ipykernel_13820\4112720750.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RISE_subset.rename(columns={


## Wind power density and potential
At https://globalwindatlas.info/api/gis/ it is possible to download for each country data regarding for example wind power density: it is especially better as while mean wind speed gives a general sense of how windy a location is but misses important variability, wind power density (or just wind density) is often considered better for energy-related decisions—especially for siting wind turbines—because it directly relates to how much usable energy is in the wind due to the fact it also includes air density (which varies by altitude, temperature and pressure).

We only dowloaded data from 100m altitude for each country present in the RISE dataset, one by one, due to the fact that selecting "The World" forces you to download around 12GB of data, including files for countries we are not interested in. All the files are in a .TIFF format, which means that it is quicker and easier to process them using a dedicated app (QGIS) instead of writing Python scripts, in order to visualize all the data as well. We uploaded them as raster files, then applied the following Python script using a internal console to colour the countries based on the following values for wind density:

Value (W/m²)	Colour (blu)

< 200	        Very light blue

200 – 400	    Light blue

400 – 600	    Medium blue

600 – 800	    Intense blue

800+	        Deep blue

Lastly we generated the PDF map with the best resolution of 600 dpi. 

In [ ]:
import os
from qgis.core import (
    QgsProject,
    QgsRasterLayer,
    QgsColorRampShader,
    QgsRasterShader,
    QgsSingleBandPseudoColorRenderer
)
from PyQt5.QtGui import QColor

# 📁 Cartella dei file .tif
folder_path = r"C:/Users/lucre/Downloads/wind power density at 100m"

for filename in os.listdir(folder_path):
    if filename.endswith(".tif"):
        file_path = os.path.join(folder_path, filename)
        layer_name = os.path.splitext(filename)[0]

        raster = QgsRasterLayer(file_path, layer_name)
        
        if raster.isValid():
            shader = QgsRasterShader()
            color_ramp = QgsColorRampShader()
            color_ramp.setColorRampType(QgsColorRampShader.Discrete)

            ramp_items = [
                QgsColorRampShader.ColorRampItem(0, QColor("#e0f3f8"), "< 200"),        # Azzurro chiarissimo
                QgsColorRampShader.ColorRampItem(200, QColor("#a6bddb"), "200–400"),    # Azzurro chiaro
                QgsColorRampShader.ColorRampItem(400, QColor("#3690c0"), "400–600"),    # Blu medio
                QgsColorRampShader.ColorRampItem(600, QColor("#0570b0"), "600–800"),    # Blu intenso
                QgsColorRampShader.ColorRampItem(800, QColor("#034e7b"), "> 800")       # Blu scurissimo
            ]

            color_ramp.setColorRampItemList(ramp_items)
            shader.setRasterShaderFunction(color_ramp)

            renderer = QgsSingleBandPseudoColorRenderer(
                raster.dataProvider(), 1, shader
            )

            raster.setRenderer(renderer)
            raster.triggerRepaint()
            QgsProject.instance().addMapLayer(raster)
            print(f"✅ Stilizzato: {layer_name}")
        else:
            print(f"❌ Non valido: {filename}")


## Photovoltaic potential and energy
The following dataset, created with the contribution of Solargis. shows data not only about the photovoltaic potential for each country on the list but also additional information on its access to electricity, reliability on supply and economic potential. We combine this dataset with the RISE one before proceeding with further analysis.

In [10]:
import pandas as pd
pv_df = pd.read_excel(r"C:\Users\lucre\Downloads\Datasets and maps\solargis_pvpotential_countryranking_2020_data.xlsx", header = 1)
pv_df.head(200)

,ISO_A3,Country or region,Note,World Bank \nRegion,"Total population, 2018","Total area, 2018",Evaluated area,Level 1 area \n(% of evaluated area),"Human development \nIndex, 2017","Gross domestic product (USD per capita), 2018",...,"Average practical potential \n(PVOUT Level 1, \nkWh/kWp/day), long-term","Average economic potential (LCOE, USD/kWh), 2018","Average PV \nseasonality index, long-term","PV equivalent area (% of total area), long-term","Cummulative installed PV capacity (MWp), 2018","Cummulative installed PV capacity (Wp per capita), 2018","Access to electricity\n(% of rural population), 2016","Electric power consumption (kWh per capita), 2014","Reliability of supply and transparency of tariff index, 2019","Approximate electricity \nTariffs for SMEs \n(US cent/kWh), 2019"
0,ABW,Aruba (Neth.),NaN,Other,105845,180.0,180,0.847926,NaN,25630.266492,...,4.9646,0.0853,1.1803,NaN,6.1,57.631442,92.452844,NaN,NaN,NaN
1,AFG,Afghanistan,NaN,SOA,37172386,652860.0,652860,0.587350,0.497695,520.896603,...,5.0159,0.0851,1.6665,NaN,22.0,0.591837,78.961074,NaN,0.0,17.6
2,AGO,Angola,NaN,AFR,30809762,1246700.0,1246700,0.751833,0.581179,3432.385736,...,4.6586,0.0919,1.3211,0.003,13.4,0.434927,15.984209,312.228825,3.0,4.6
3,ALB,Albania,NaN,ECA,2866376,27400.0,27400,0.614858,0.784911,5253.630064,...,4.0426,0.1051,2.3266,0.210,1.0,0.348873,100.000000,2309.366503,3.0,8.7
4,AND,Andorra,NaN,Other,77006,470.0,470,0.110961,0.857684,42029.762737,...,4.1990,0.0986,2.1531,NaN,0.0,0.000000,100.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,USA,United States of America,up to parallel 60°N,Other,327167434,9147420.0,8039753,0.663291,0.923914,62641.014570,...,4.3584,0.1028,1.8847,0.425,49692.0,151.885533,100.000000,12993.961825,7.2,17.1
196,UZB,Uzbekistan,NaN,ECA,32955400,425400.0,425400,0.484091,0.709847,1532.371639,...,4.2688,0.0997,2.4776,0.107,3.9,0.118342,100.000000,1645.441629,8.0,5.7
197,VCT,Saint Vincent and the Grenadines,NaN,LCR,110210,390.0,390,0.500000,0.722708,7377.678832,...,4.2728,0.1025,1.2374,NaN,1.3,11.795663,100.000000,NaN,0.0,31.5
198,VEN,R. B. de Venezuela,NaN,LCR,28870195,882050.0,882050,0.459566,0.760773,NaN,...,4.3545,0.0977,1.2605,0.039,4.0,0.138551,96.384375,2718.942775,0.0,NaN


In [8]:
pv_df['Country or region'] = pv_df['Country or region'].str.strip()
RISE_df['Country'] = RISE_df['Country'].str.strip()


In [11]:
pv_countries = pv_df['Country or region'].unique()
print(pv_countries)

['Aruba (Neth.)' 'Afghanistan' 'Angola' 'Albania' 'Andorra'
 'United Arab Emirates' 'Argentina' 'Armenia' 'American Samoa (U.S.)'
 'Antigua and Barbuda' 'Australia' 'Austria' 'Azerbaijan' 'Burundi'
 'Belgium' 'Benin' 'Burkina Faso' 'Bangladesh' 'Bulgaria' 'Bahrain'
 'The Bahamas' 'Bosnia and Herzegovina' 'Belarus' 'Belize'
 'Bermuda (U.K.)' 'Bolivia' 'Brazil' 'Barbados' 'Brunei Darussalam'
 'Bhutan' 'Botswana' 'Central African Republic' 'Canada' 'Switzerland'
 'Chile' 'China' "Côte d'Ivoire" 'Cameroon' 'Democratic Republic of Congo'
 'Congo' 'Colombia' 'Comoros' 'Cabo Verde' 'Costa Rica' 'Cuba'
 'Curaçao (Neth.)' 'Cayman Islands (U.K.)' 'Cyprus' 'Czech Republic'
 'Germany' 'Djibouti' 'Dominica' 'Denmark' 'Dominican Republic' 'Algeria'
 'Ecuador' 'Arab Republic of Egypt' 'Eritrea' 'Spain' 'Estonia' 'Ethiopia'
 'Fiji' 'France' 'Federated States of Micronesia' 'Gabon' 'United Kingdom'
 'Georgia' 'Ghana' 'Guinea' 'The Gambia' 'Guinea-Bissau'
 'Equatorial Guinea' 'Greece' 'Grenada' 'Guatema

In [12]:
RISE_countries = RISE_df['Country'].unique()
print(RISE_countries)

['Afghanistan' 'Algeria' 'Angola' 'Argentina' 'Armenia' 'Australia'
 'Austria' 'Bahrain' 'Bangladesh' 'Belarus' 'Belgium' 'Benin' 'Bolivia'
 'Brazil' 'Burkina Faso' 'Burundi' 'Cambodia' 'Cameroon' 'Canada'
 'Central African Republic' 'Chad' 'Chile' 'China' 'Colombia'
 'Congo, Dem. Rep.' 'Congo, Rep.' "Côte d'Ivoire" 'Czech Republic'
 'Denmark' 'Dominican Republic' 'Ecuador' 'Egypt, Arab Rep.' 'Eritrea'
 'Ethiopia' 'Finland' 'France' 'Germany' 'Ghana' 'Greece' 'Guatemala'
 'Guinea' 'Haiti' 'Honduras' 'India' 'Indonesia' 'Iran, Islamic Rep.'
 'Italy' 'Japan' 'Jordan' 'Kazakhstan' 'Kenya' 'Korea, Rep.' 'Kuwait'
 'Kyrgyz Republic' 'Lao PDR' 'Lebanon' 'Liberia' 'Madagascar' 'Malawi'
 'Malaysia' 'Maldives' 'Mali' 'Mauritania' 'Mexico' 'Mongolia' 'Morocco'
 'Mozambique' 'Myanmar' 'Nepal' 'Netherlands' 'Nicaragua' 'Niger'
 'Nigeria' 'Pakistan' 'Peru' 'Philippines' 'Poland' 'Qatar' 'Romania'
 'Russian Federation' 'Rwanda' 'Saudi Arabia' 'Senegal' 'Sierra Leone'
 'Solomon Islands' 'Somalia' 'Sou

In [14]:
country_name_map = {
    "United States": "United States of America",
    "Egypt, Arab Rep.": "Arab Republic of Egypt",
    "Venezuela, RB": "R. B. de Venezuela",
    "Iran, Islamic Rep.": "Islamic Republic of Iran",
    "Yemen, Rep.": "Republic of Yemen",
    "Congo, Dem. Rep.": "Democratic Republic of Congo",
    "Congo, Rep.": "Congo",
    "Lao PDR": "Lao People's Democratic Republic",
    "Korea, Rep.": "Republic of Korea"
}


In [17]:
RISE_df['Country'] = RISE_df['Country'].replace(country_name_map)
filtered_pv_df = pv_df[pv_df['Country or region'].isin(RISE_df['Country'])]
merged_df = pd.merge(filtered_pv_df, RISE_df, left_on='Country or region', right_on='Country', how='inner')


In [18]:
pv_countries = set(pv_df['Country or region'])
rise_countries = set(RISE_df['Country'])

missing_in_rise = pv_countries - rise_countries
print("❌ Countries in PV data but NOT in RISE data:")
print(sorted(missing_in_rise))


❌ Countries in PV data but NOT in RISE data:
['Albania', 'American Samoa (U.S.)', 'Andorra', 'Antigua and Barbuda', 'Aruba (Neth.)', 'Azerbaijan', 'Barbados', 'Belize', 'Bermuda (U.K.)', 'Bhutan', 'Bosnia and Herzegovina', 'Botswana', 'British Virgin Islands (U.K.)', 'Brunei Darussalam', 'Bulgaria', 'Cabo Verde', 'Cayman Islands (U.K.)', 'Comoros', 'Costa Rica', 'Croatia', 'Cuba', 'Curaçao (Neth.)', 'Cyprus', 'D. P. R. of Korea', 'Djibouti', 'Dominica', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Eswatini', 'Federated States of Micronesia', 'Fiji', 'French Polynesia (Fr.)', 'Gabon', 'Georgia', 'Grenada', 'Guam (U.S.)', 'Guinea-Bissau', 'Guyana', 'Hong Kong, SAR', 'Hungary', 'Iraq', 'Ireland', 'Isle of Man (U.K.)', 'Israel', 'Jamaica', 'Kiribati', 'Kosovo', 'Latvia', 'Lesotho', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macau, SAR', 'Malta', 'Marshall Islands', 'Mauritius', 'Moldova', 'Montenegro', 'Namibia', 'Nauru', 'New Caledonia (Fr.)', 'New Zealand', 'North Macedonia